## acquire

Goal: Predict the logerror

Zillow data:
- 2017 data
- Latest transaction per property id only.
- The logerror from that latest transaction.
- All fields related to the properties.
- Gather descriptions from the lookup tables.
- Only properties with latitude and longitude.
- Only single family homes.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, LabelEncoder, OneHotEncoder, QuantileTransformer, MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings("ignore")

import acquire
import summarize
import prepare

In [7]:
df = acquire.get_zillow_data()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85224 entries, 0 to 85223
Data columns (total 60 columns):
id                              85224 non-null int64
parcelid                        85224 non-null int64
airconditioningtypeid           28311 non-null float64
architecturalstyletypeid        261 non-null float64
basementsqft                    42 non-null float64
bathroomcnt                     85224 non-null float64
bedroomcnt                      85224 non-null float64
buildingclasstypeid             0 non-null object
buildingqualitytypeid           53810 non-null float64
calculatedbathnbr               85207 non-null float64
decktypeid                      654 non-null float64
finishedfloor1squarefeet        6821 non-null float64
calculatedfinishedsquarefeet    85224 non-null float64
finishedsquarefeet12            85035 non-null float64
finishedsquarefeet13            2 non-null float64
finishedsquarefeet15            29 non-null float64
finishedsquarefeet50            682

Only single family

In [8]:
df = df[df.propertylandusetypeid == 261]

## prepare 

### Missing Values

- remove columns with > 99% missing and rows  > 40% missing
- aggregate pool information: use all pool and spa columns to compute a single boolean attribute of `has_pool`
- fill with 0: taxdelinquencyflag, fireplacecnt, garagecarcnt and convert them to boolean
- After doing all that, then remove all columns with > 5% missing, and following that, rows with > 99% missing

In [11]:
# remove columns with > 99% missing and rows  > 40% missing
df = prepare.handle_missing_values(df, prop_required_column = .01, prop_required_row = .40)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58419 entries, 0 to 85221
Data columns (total 31 columns):
id                              58419 non-null int64
parcelid                        58419 non-null int64
bathroomcnt                     58419 non-null float64
bedroomcnt                      58419 non-null float64
calculatedbathnbr               58419 non-null float64
calculatedfinishedsquarefeet    58419 non-null float64
finishedsquarefeet12            58419 non-null float64
fips                            58419 non-null float64
fullbathcnt                     58419 non-null float64
latitude                        58419 non-null float64
longitude                       58419 non-null float64
lotsizesquarefeet               58419 non-null float64
propertycountylandusecode       58419 non-null object
propertylandusetypeid           58419 non-null float64
rawcensustractandblock          58419 non-null float64
regionidcity                    58419 non-null float64
regionidcounty  

In [12]:
# aggregate pool information: use all pool and spa columns to compute a single attribute of pool_spa
# # gather pool columns
# pool_cols = ['hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid2', 'pooltypeid7']
# # fill all missing values with 0
# pool = df[pool_cols].fillna(0)
# # where there is a value in one or more of the pool attributes, assign a 1 to a new col named 'pool'
# pool.loc[pool.sum(axis=1)>0, 'has_pool'] = 1
# # append the new column to our original dataframe and remove the original pool columns 
# df = df.join(pool[['has_pool']])

# fill with 0
df.loc[df.taxdelinquencyflag == 'Y', 'is_taxdelinquent'] = 1
df.loc[df.fireplacecnt > 0, 'has_fireplace'] = 1
df.loc[df.garagecarcnt > 0, 'has_garage'] = 1
fill_with_0 = ['has_garage', 'has_fireplace', 'has_pool', 'is_taxdelinquent']
df[fill_with_0] = df[fill_with_0].fillna(0)

# remove columns where > 5% missing and rows where > 99% missing
df = prepare.handle_missing_values(df, prop_required_column = .95, prop_required_row = .99)

KeyError: "None of [Index(['hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid2',\n       'pooltypeid7'],\n      dtype='object')] are in the [columns]"

### Variable Changes

Are there any instances where taxvaluedollarcnt is not equal to the sum of the land tax value and the structure tax value? (landtaxvaluedollarcnt + structuretaxvaluedollarcnt). 

In [ ]:
np.where((df.taxvaluedollarcnt - (df.landtaxvaluedollarcnt + df.structuretaxvaluedollarcnt)) != 0)
# add taxvaluedollarcnt to list to drop

No. I will attempt to reduce the dependency between variables and extracting the most unique information from each. 

- `land_dollar_per_sqft`: a land dollar per sqft (landtaxvaluedollarcnt/lotsizesquarefeet)
- `structure_dollar_per_sqft`: structuretaxvaluedollarcnt/calculatedfinishedsquarefeet
- `tax_rate`: taxvaluedollarcnt/taxamount
- compute `living_area_sqft` by subtracting estimated square feet from bedrooms (121: 11x11) and bathrooms (36: 6x6)
- compute `bedbath_index` where multiple bedrooms by a weight of 2, full baths by weight of 1, half/three-quarter baths by weight of .5, then sum them all together.  


In [ ]:
df['structure_dollar_per_sqft'] = df.structuretaxvaluedollarcnt/df.calculatedfinishedsquarefeet
df['land_dollar_per_sqft'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df['living_area_sqft'] = df.calculatedfinishedsquarefeet - (df.bedroomcnt*121 + df.bathroomcnt*36)
df['tax_rate'] = df.taxvaluedollarcnt/df.taxamount
df['bedbath_index'] = df.bedroomcnt*2 + df.fullbathcnt + .5*(df.bathroomcnt-df.fullbathcnt)

- turn yearbuilt into age (from present)
- reduce regionidcity into the top 5 cities and the others assign to a catch-all id.
- take the first 3 digits of zip to reduce the variance in zipcode 
- Look at variables that don't actually represent numeric values to think about encoding. (fips, regionidcity, regionidzip, regionidcounty)

In [ ]:
df['age'] = 2017 - df.yearbuilt

In [ ]:
df.loc[:,'latitude'] = df.loc[:,'latitude']/1e6
df.loc[:,'longitude'] = df.loc[:,'longitude']/1e6

City ID and County: Is there any cross-over or is city purely a subset of county? 

Count the number of counties each city is located in:

In [ ]:
ct = pd.DataFrame(pd.crosstab(df.regionidcity, df.regionidcounty))
s = ct.astype(bool).sum(axis=1)
s = s.where(s>1).dropna()
pd.crosstab(df[df.regionidcity.isin(list(s.index))].regionidcity, df[df.regionidcity.isin(list(s.index))].regionidcounty)

Taking a look at these, I can see that when there are multiple counties, there is clearly a dominant county and only a handful of properties in the other. I will 'fix' the anomalies to be in what is likely the correct county. I'll test it here, but will need to implement above before we do all the prepping. 

In [ ]:
df.loc[df.regionidcity.isin([5465.0,12447.0,12520.0]), 'regionidcounty'] = 3101.0
df.loc[df.regionidcity.isin([10608.0,15237.0,18874.0,44833.0]), 'regionidcounty'] = 1286.0
df.loc[df.regionidcity==41673.0, 'regionidcounty'] = 2061.0
df.regionidcounty.value_counts()

- Looking at the counts for each county, it seems reasonable to not split county 2061 geographically much more than that.  
- County 1286 can probably be split more effectively: city 16764 and all others 
- County 3101 can definitedly be split more effectively: city 12447, 5534, 46298, 40227, and all others

However, I'm going to wait to do this. I will first run some statistical tests to see if there are cities and zips that have significantly different logerror from the rest of the properties. 

In [ ]:
# df.loc[(df['regionidcity']==12447) | (df['regionidcity']==5534) | (df['regionidcity']==40227) | (df['regionidcity']==46298) | (df['regionidcity']==16764), 'cityid'] = df['regionidcity']
# df.cityid.fillna(0, inplace=True)

In [ ]:
# have to do float first because of an issue with 0.0, then int, then string to ensure no decimals in the string.  
# df['cityid'] = df.cityid.astype(float).astype(int).astype(str)
df['regionidcity'] = df.regionidcity.astype(float).astype(int).astype(str)
df['regionidzip'] = df.regionidzip.astype(float).astype(int).astype(str)
df['regionidcounty'] = df.regionidcounty.astype(float).astype(int).astype(str)

In [ ]:
# df['loc_id'] = df.regionidcounty + '_' + df.cityid

In [ ]:
# clean up remaining columns
df_prepped = df.drop(columns=(['id','parcelid','assessmentyear','propertycountylandusecode',
                               'propertylandusedesc','transactiondate', 'propertylandusetypeid',
                               'finishedsquarefeet12', 'taxvaluedollarcnt', 'fips',
                               'yearbuilt', 'rawcensustractandblock', 'censustractandblock', 'roomcnt', 
                               'calculatedbathnbr','taxamount','calculatedfinishedsquarefeet',
                               'landtaxvaluedollarcnt','structuretaxvaluedollarcnt',
                               'bedroomcnt','bathroomcnt','fullbathcnt'
                              ]))

In [ ]:
df_prepped.shape

### Split Data

In [ ]:
train, test = train_test_split(df_prepped, test_size=.30)

### Scaling

#### Create Uniform Scaler
when we the space between doesn't matter as much as order does, a uniform scaler is a good choice. 

In [ ]:
def scale_uniform(train, test, column_list):
    scaler = QuantileTransformer(output_distribution='uniform', random_state=123)
    train_scaled = pd.DataFrame(scaler.fit_transform(train[column_list]), 
                                columns = column_list, 
                                index = train.index)
    train.drop(columns=column_list, inplace=True)
    train = train.join(train_scaled)
    
    test_scaled = pd.DataFrame(scaler.transform(test[column_list]), 
                                columns = column_list, 
                                index = test.index)
    test.drop(columns=column_list, inplace=True)
    test = test.join(test_scaled)
    
    return train, test, scaler

#### Create MinMax Scaler
When we want to preserve distance but want to be within bounds, a min-max scaler is a good choice. 

In [ ]:
def scale_minmax(train, test, column_list):
    scaler = MinMaxScaler(feature_range=(0,1))
    train_scaled = pd.DataFrame(scaler.fit_transform(train[column_list]), 
                                columns = column_list, 
                                index = train.index)
    train.drop(columns=column_list, inplace=True)
    train = train.join(train_scaled)
    
    test_scaled = pd.DataFrame(scaler.transform(test[column_list]), 
                                columns = column_list, 
                                index = test.index)
    test.drop(columns=column_list, inplace=True)
    test = test.join(test_scaled)
    
    return train, test, scaler

We will scale square feet, dollar per square foot, tax rate, beds & baths using a uniform scaler as that will help minimize the impact extreme outliers will have. 

For latitude, longitude and age, we want to preserve the distance between. We want 1876 to be futher away from the next oldest house of 1900 than 1900 is from the next oldest house of 1901. For this reason, we will use a min-max scaler. 

In [ ]:
column_list1 = ['lotsizesquarefeet','structure_dollar_per_sqft','land_dollar_per_sqft','living_area_sqft','tax_rate','bedbath_index']
train, test, scaler_uniform = scale_uniform(train, test, column_list1)

column_list2 = ['latitude','longitude','age']
train, test, scaler_minmax = scale_minmax(train, test, column_list2)

In [ ]:
# train_prepped.describe().T
train.info()

## Cluster

### K-Means

#### Elbow Method to determine best 'K'

1. Let's first cluster by geolocation of latitude and longitude along with dollar per square foot (land and structure) and tax rate.

2. Then we will cluster by lot size, living area, beds and baths, and age. 

##### Clustering 1: location, dollar/sqft, tax rate

In [ ]:
cluster1_cols = ['latitude','longitude','land_dollar_per_sqft','structure_dollar_per_sqft']

Compute and plot the sum squared distances of each sample to closest cluster center at each k-value.  

In [ ]:
def select_k(cluster_df, ks):
    sse = []
    for k in ks:
        kmeans = KMeans(n_clusters=k, n_init=1, max_iter=100, random_state=123)
        kmeans.fit(cluster_df)

        # inertia: Sum of squared distances of samples to their closest cluster center.
        sse.append(kmeans.inertia_)

    # print(pd.DataFrame(dict(k=ks, sse=sse)))

    p = plt.plot(ks, sse, 'bx-')
    p = plt.xlabel('k')
    p = plt.ylabel('SSE')
    p = plt.title('The Elbow Method to find the optimal k')

    compare_df = pd.DataFrame(dict(k=ks, sse=sse)).assign(change_in_sse=lambda df: df.sse.diff())
    return compare_df, p

In [ ]:
select_k(cluster_df = train[cluster1_cols], ks = range(1,13))

I would say 6 or 8 is hwere the bottom of the elbow sits.  
Let's compare k=6 vs. k=8. 

In [ ]:
def compare_clusters(cluster_df, x_column, y_column, z_column, k1, k2):
    estimators = [(str(k1)+' Clusters', KMeans(n_clusters=k1, n_init=1, max_iter=100, random_state=123)),
                  (str(k2)+' Clusters', KMeans(n_clusters=k2, n_init=1, max_iter=100, random_state=123))]
    
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), subplot_kw={'projection': '3d'})
    
    for ax, (title, kmeans) in zip(axs, estimators):
        # fit the kmeans object
        kmeans.fit(cluster_df)
        
        labels = kmeans.labels_
        
        ax.scatter(cluster_df[x_column], 
                   cluster_df[y_column],
                   cluster_df[z_column],
                   c=labels.astype(np.float), edgecolor='k')
        ax.set(xticklabels=[], yticklabels=[], zticklabels=[])
        ax.set(xlabel=x_column, ylabel=y_column, zlabel=z_column)
        ax.set(title=title)
    
    plt.show()

In [ ]:
compare_clusters(cluster_df=train[cluster1_cols],
                x_column='latitude', y_column='land_dollar_per_sqft', z_column='longitude',
                k1=6, k2=8)

In [ ]:
compare_clusters(cluster_df=train[cluster1_cols],
                x_column='latitude', y_column='structure_dollar_per_sqft', z_column='longitude',
                k1=6, k2=8)

In [ ]:
compare_clusters(cluster_df=train[cluster1_cols],
                x_column='latitude', y_column='structure_dollar_per_sqft', z_column='land_dollar_per_sqft',
                k1=6, k2=8)

As latitude moves east (left on the chart), we can see the land dollar per sqft and structure dollar per sqft increases, indicated by the slope upward as you move back and to the left. (low latitude, high structure dollar per sqft, high land dollar per sqft. 

I'm going to go with 8.

In [ ]:
def create_k_clusters(train, test, cluster_feature_id, cluster_cols, k):
    kmeans = KMeans(n_clusters=k, n_init=1, max_iter=100, random_state=123)
    kmeans.fit(train[cluster_cols])
    cluster_feature = 'cluster'+str(cluster_feature_id)+'_id'
    train[cluster_feature] = kmeans.predict(train[cluster_cols])
    test[cluster_feature] = kmeans.predict(test[cluster_cols])
    return train, test, kmeans

In [ ]:
train, test, kmeans1 = create_k_clusters(train, test, cluster_feature_id = 1, cluster_cols = cluster1_cols, k=8)

In [ ]:
train.rename(index=str, columns={'cluster1_id': 'cluster_loc'}, inplace=True)
test.rename(index=str, columns={'cluster1_id': 'cluster_loc'}, inplace=True)

In [ ]:
train['cluster'] = kmeans1.labels_
train.cluster = 'cluster_' + (train.cluster + 1).astype('str')
for i in range(0,len(cluster1_cols)):
    for j in range(0, len(cluster1_cols)):
        sns.relplot(data=train, x=cluster1_cols[i], y=cluster1_cols[j], hue='cluster')

In [ ]:
train.cluster_loc.value_counts()

In [ ]:
test.cluster_loc.value_counts()

##### Clustering 2: size fields and age

In [ ]:
cluster2_cols = ['lotsizesquarefeet', 'living_area_sqft', 'bedbath_index','age']

In [ ]:
select_k(cluster_df = train[cluster2_cols], ks = range(1,13))

Compare 5 vs. 7 clusters

In [ ]:
compare_clusters(cluster_df=train[cluster2_cols],
                x_column='lotsizesquarefeet', y_column='living_area_sqft', z_column='bedbath_index',
                k1=5, k2=7)

In [ ]:
compare_clusters(cluster_df=train[cluster2_cols],
                x_column='lotsizesquarefeet', y_column='living_area_sqft', z_column='age',
                k1=5, k2=7)

In [ ]:
compare_clusters(cluster_df=train[cluster2_cols],
                x_column='lotsizesquarefeet', y_column='bedbath_index', z_column='age',
                k1=5, k2=7)

In [ ]:
compare_clusters(cluster_df=train[cluster2_cols],
                x_column='living_area_sqft', y_column='bedbath_index', z_column='age',
                k1=5, k2=7)

I'm going to go with 7 clusters.

In [ ]:
train, test, kmeans2 = create_k_clusters(train, test, cluster_feature_id = 2, cluster_cols = cluster2_cols, k=7)

In [ ]:
train.rename(index=str, columns={'cluster2_id': 'cluster_home'}, inplace=True)
test.rename(index=str, columns={'cluster2_id': 'cluster_home'}, inplace=True)

In [ ]:
train['cluster'] = kmeans2.labels_
train.cluster = 'cluster_' + (train.cluster + 1).astype('str')

for i in range(0,len(cluster2_cols)):
    for j in range(0, len(cluster2_cols)):
        sns.relplot(data=train, x=cluster2_cols[i], y=cluster2_cols[j], hue='cluster')

### Feature Selection

#### Which variables and clusters give information about logerror?

First, I need to confirm that 'logerror' is normally distributed, to determine if I can run t-tests to test the differences in means across different clusters. 

In [ ]:
plt.hist(train.logerror, bins=1000)
plt.show()

Looks good!

**Test:** Home driven clusters

First, let's look at the mean log error by cluster id. We will do this for both the train and test as more of a data quality check...to confirm that our cluster id's are showing similar results in both samples. 

In [ ]:
print(pd.DataFrame(train.groupby(['cluster_home'])['logerror'].mean().reset_index()))
print(pd.DataFrame(test.groupby(['cluster_home'])['logerror'].mean().reset_index()))

Test the different in the mean logerror for each cluster vs all others. When the p-value is < .05 then we keep the cluster id, else we replace the cluster id with -1. 

In [ ]:
import scipy as sp
import numpy as np

less_significant_clusters = []

for i in range(0,max(train.cluster_home)+1):
    stat, pval = sp.stats.ttest_ind(
        train[train.cluster_home == i].logerror.dropna(),
        train[train.cluster_home != i].logerror.dropna())
    if pval > .05:
        less_significant_clusters = less_significant_clusters + [i]

In [ ]:
train.cluster_home = train.cluster_home.replace(less_significant_clusters, -1)
test.cluster_home = test.cluster_home.replace(less_significant_clusters, -1)

In [ ]:
test.cluster_home.value_counts()

In [ ]:
train.cluster_home.value_counts()

**Test:** Location driven clusters

In [ ]:
pd.DataFrame(train.groupby(['cluster_loc'])['logerror'].mean().reset_index())

In [ ]:
less_significant_clusters = []

for i in range(0,max(train.cluster_loc)+1):
    stat, pval = sp.stats.ttest_ind(
        train[train.cluster_loc == i].logerror.dropna(),
        train[train.cluster_loc != i].logerror.dropna())
    if pval > .05:
        less_significant_clusters = less_significant_clusters + [i]

In [ ]:
less_significant_clusters

In [ ]:
train.cluster_loc = train.cluster_loc.replace(less_significant_clusters, -1)
test.cluster_loc = test.cluster_loc.replace(less_significant_clusters, -1)

**Test:** is_taxdelinquent

In [ ]:
pd.DataFrame(train.groupby(['is_taxdelinquent'])['logerror'].mean().reset_index())

In [ ]:
stats, pval = sp.stats.ttest_ind(
    train[train.is_taxdelinquent == 0].logerror.dropna(),
    train[train.is_taxdelinquent == 1].logerror.dropna())

if pval > .05:
    train.drop(columns=['is_taxdelinquent'], inplace=True)
    test.drop(columns=['is_taxdelinquent'], inplace=True)

**Data Validation**

In [ ]:
print(pval)
# if pval < 0.05 then the column should still exist: 
'is_taxdelinquent' in train.columns

**Results:** is_taxdelinquent was correctly left as feature. 

_________________________

**Test:** has_pool

In [ ]:
pd.DataFrame(train.groupby(['has_pool'])['logerror'].mean().reset_index())

In [ ]:
stats, pval = sp.stats.ttest_ind(
    train[train.has_pool == 0].logerror.dropna(),
    train[train.has_pool == 1].logerror.dropna())

if pval > .05:
    train.drop(columns=['has_pool'], inplace=True)
    test.drop(columns=['has_pool'], inplace=True)

**Data Validation**

In [ ]:
print(pval)
# verify column exists if pval < 0.05, and not if greater
'has_pool' in train.columns

**Results:** has_pool was correctly left as feature

____________________________ 

**Test:** has_fireplace

In [ ]:
pd.DataFrame(train.groupby(['has_fireplace'])['logerror'].mean().reset_index())

In [ ]:
stats, pval = sp.stats.ttest_ind(
    train[train.has_fireplace == 0].logerror.dropna(),
    train[train.has_fireplace == 1].logerror.dropna())

if pval > .05:
    train.drop(columns=['has_fireplace'], inplace=True)
    test.drop(columns=['has_fireplace'], inplace=True)

**Data Validation**

In [ ]:
print(pval)
# verify column exists if pval < 0.05, and not if greater
'has_fireplace' in train.columns

**Results:** has_fireplace was correctly removed
_______________________

**Test:** has_garage

In [ ]:
pd.DataFrame(train.groupby(['has_garage'])['logerror'].mean().reset_index())

In [ ]:
stats, pval = sp.stats.ttest_ind(
    train[train.has_garage == 0].logerror.dropna(),
    train[train.has_garage == 1].logerror.dropna())

if pval > .05:
    train.drop(columns=['has_garage'], inplace=True)
    test.drop(columns=['has_garage'], inplace=True)

**Data Validation**

In [ ]:
print(pval)

# verify column exists if pval < 0.05, and not if greater
'has_garage' in train.columns


**Results:** has_garage was correctly left as a feature

_______________________

**Clean up remaining features**

In [ ]:
train.head()

In [ ]:
cols_to_remove = ['tax_rate','regionidcity','regionidzip']
non_cluster_features = ['lotsizesquarefeet', 'structure_dollar_per_sqft', 'land_dollar_per_sqft',
                        'living_area_sqft', 'bedbath_index', 'latitude', 'longitude', 'age']
train_no_clusters = train[non_cluster_features+['regionidcounty','logerror']]
test_no_clusters = test[non_cluster_features+['regionidcounty','logerror']]

In [ ]:
cols_to_remove = cols_to_remove + non_cluster_features
train_clusters = train.drop(columns=cols_to_remove)
test_clusters = test.drop(columns=cols_to_remove)

In [ ]:
train_no_clusters.head()

In [ ]:
train_clusters.head()

### Encode

Which columns are of numeric format but represent classes or categories?
fips
rawcensustractandblock
regionidcity
regionidcounty
regionidzip


In [ ]:
def encode(train, test, col_name):
  
    encoded_values = sorted(list(train[col_name].unique()))
    columns = [col_name + '_' + str(val) for val in encoded_values]

    # Integer Encoding
    int_encoder = LabelEncoder()
    train.encoded = int_encoder.fit_transform(train[col_name])
    test.encoded = int_encoder.transform(test[col_name])

    # create 2D np arrays of the encoded variable (in train and test)
    train_array = np.array(train.encoded).reshape(len(train.encoded),1)
    test_array = np.array(test.encoded).reshape(len(test.encoded),1)
    
    # One Hot Encoding
    ohe = OneHotEncoder(sparse=False, categories='auto')
    train_ohe = ohe.fit_transform(train_array)
    test_ohe = ohe.transform(test_array)

    # Turn the array of new values into a data frame with columns names being the values
    # and index matching that of train/test
    # then merge the new dataframe with the existing train/test dataframe
    train_encoded = pd.DataFrame(data=train_ohe,
                            columns=columns, index=train.index)
    train = train.join(train_encoded)

    test_encoded = pd.DataFrame(data=test_ohe,
                               columns=columns, index=test.index)
    test = test.join(test_encoded)

    return train, test, int_encoder, ohe

def one_hot_encode(train, test, col_name):
  
    encoded_values = sorted(list(train[col_name].unique()))
    columns = [col_name + '_' + str(val) for val in encoded_values]

    # create 2D np arrays of the encoded variable (in train and test)
    train_array = np.array(train[col_name]).reshape(len(train[col_name]),1)
    test_array = np.array(test[col_name]).reshape(len(test[col_name]),1)
    
    # One Hot Encoding
    ohe = OneHotEncoder(sparse=False, categories='auto')
    train_ohe = ohe.fit_transform(train_array)
    test_ohe = ohe.transform(test_array)

    # Turn the array of new values into a data frame with columns names being the values
    # and index matching that of train/test
    # then merge the new dataframe with the existing train/test dataframe
    train_encoded = pd.DataFrame(data=train_ohe,
                            columns=columns, index=train.index)
    train = train.join(train_encoded)

    test_encoded = pd.DataFrame(data=test_ohe,
                               columns=columns, index=test.index)
    test = test.join(test_encoded)

    return train, test, ohe

In [ ]:
train_clusters, test_clusters, ohe_loc = one_hot_encode(train_clusters, test_clusters, 'cluster_loc')
train_clusters, test_clusters, ohe_home = one_hot_encode(train_clusters, test_clusters, 'cluster_home')

Build 3 models, 1 for each county
Try with clusters and then try with original features

In [ ]:
train_clusters.regionidcounty.value_counts()

Separate the clusters dataframes by county

In [ ]:
train_3101_c = train_clusters[train_clusters.regionidcounty=='3101']
test_3101_c = test_clusters[test_clusters.regionidcounty=='3101']

train_1286_c = train_clusters[train_clusters.regionidcounty=='1286']
test_1286_c = test_clusters[test_clusters.regionidcounty=='1286']

train_2061_c = train_clusters[train_clusters.regionidcounty=='2061']
test_2061_c = test_clusters[test_clusters.regionidcounty=='2061']

Separate the non-clusters dataframes by county

In [ ]:
train_3101_nc = train_no_clusters[train_no_clusters.regionidcounty=='3101']
test_3101_nc = test_no_clusters[test_no_clusters.regionidcounty=='3101']

train_1286_nc = train_no_clusters[train_no_clusters.regionidcounty=='1286']
test_1286_nc = test_no_clusters[test_no_clusters.regionidcounty=='1286']

train_2061_nc = train_no_clusters[train_no_clusters.regionidcounty=='2061']
test_2061_nc = test_no_clusters[test_no_clusters.regionidcounty=='2061']

Now that we have encoded and split by county, we can remove some other columns to have our final X with features.
We only need a y_train and y_test for each county, not separated by the features, obviously. 

X dataframes for the cluster features:

In [ ]:
cols_to_drop = ['cluster_loc', 'cluster_home', 'cluster_loc_-1', 'cluster_home_-1', 'logerror', 'regionidcounty']

X_train_3101_c = train_3101_c.drop(columns=cols_to_drop)
X_test_3101_c = test_3101_c.drop(columns=cols_to_drop)

X_train_1286_c = train_1286_c.drop(columns=cols_to_drop)
X_test_1286_c = test_1286_c.drop(columns=cols_to_drop)

X_train_2061_c = train_2061_c.drop(columns=cols_to_drop)
X_test_2061_c = test_2061_c.drop(columns=cols_to_drop)

X dataframes for the non-cluster features

In [ ]:
cols_to_drop = ['logerror', 'regionidcounty']

X_train_3101_nc = train_3101_nc.drop(columns=cols_to_drop)
X_test_3101_nc = test_3101_nc.drop(columns=cols_to_drop)

X_train_1286_nc = train_1286_nc.drop(columns=cols_to_drop)
X_test_1286_nc = test_1286_nc.drop(columns=cols_to_drop)

X_train_2061_nc = train_2061_nc.drop(columns=cols_to_drop)
X_test_2061_nc = test_2061_nc.drop(columns=cols_to_drop)

y dataframes

In [ ]:
y_train_3101 = train_3101_c[['logerror']]
y_test_3101 = test_3101_c[['logerror']]

y_train_1286 = train_1286_c[['logerror']]
y_test_1286 = test_1286_c[['logerror']]

y_train_2061 = train_2061_c[['logerror']]
y_test_2061 = test_2061_c[['logerror']]

## Model

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor, LassoCV
from sklearn.tree import DecisionTreeRegressor

### county: 3101

#### Baseline

In [ ]:
se = y_train_3101.logerror * y_train_3101.logerror
mse = se.mean()
rmse = mse**1/2
rmse

#### Clustering Features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = regr.predict(X_train_3101_c)
print(mean_squared_error(y_train_3101, y_pred_3101)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = sgd.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = lasso.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = dt.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

#### non-clustering features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = regr.predict(X_train_3101_nc)
print(mean_squared_error(y_train_3101, y_pred_3101)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = sgd.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = lasso.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = dt.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

WOW!!!

### county: 3101

#### Baseline

In [ ]:
se = y_train_3101.logerror * y_train_3101.logerror
mse = se.mean()
rmse = mse**1/2
rmse

#### Clustering Features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = regr.predict(X_train_3101_c)
print(mean_squared_error(y_train_3101, y_pred_3101)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = sgd.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = lasso.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_3101_c, y_train_3101)
y_pred_3101 = dt.predict(X_train_3101_c)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

#### non-clustering features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = regr.predict(X_train_3101_nc)
print(mean_squared_error(y_train_3101, y_pred_3101)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = sgd.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = lasso.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_3101_nc, y_train_3101)
y_pred_3101 = dt.predict(X_train_3101_nc)
mean_squared_error(y_train_3101, y_pred_3101)**1/2

WOW!!!

### county: 1286

#### Baseline

In [ ]:
se = y_train_1286.logerror * y_train_1286.logerror
mse = se.mean()
rmse = mse**1/2
rmse

#### Clustering Features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_1286_c, y_train_1286)
y_pred_1286 = regr.predict(X_train_1286_c)
print(mean_squared_error(y_train_1286, y_pred_1286)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_1286_c, y_train_1286)
y_pred_1286 = sgd.predict(X_train_1286_c)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_1286_c, y_train_1286)
y_pred_1286 = lasso.predict(X_train_1286_c)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_1286_c, y_train_1286)
y_pred_1286 = dt.predict(X_train_1286_c)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

#### non-clustering features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_1286_nc, y_train_1286)
y_pred_1286 = regr.predict(X_train_1286_nc)
print(mean_squared_error(y_train_1286, y_pred_1286)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_1286_nc, y_train_1286)
y_pred_3101 = sgd.predict(X_train_1286_nc)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_1286_nc, y_train_1286)
y_pred_1286 = lasso.predict(X_train_1286_nc)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_1286_nc, y_train_1286)
y_pred_1286 = dt.predict(X_train_1286_nc)
mean_squared_error(y_train_1286, y_pred_1286)**1/2

WOW again!

### county: 2061

#### Baseline

In [ ]:
se = y_train_2061.logerror * y_train_2061.logerror
mse = se.mean()
rmse = mse**1/2
rmse

#### Clustering Features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_2061_c, y_train_2061)
y_pred_2061 = regr.predict(X_train_2061_c)
print(mean_squared_error(y_train_2061, y_pred_2061)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_2061_c, y_train_2061)
y_pred_2061 = sgd.predict(X_train_2061_c)
mean_squared_error(y_train_2061, y_pred_2061)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_2061_c, y_train_2061)
y_pred_2061 = lasso.predict(X_train_2061_c)
mean_squared_error(y_train_2061, y_pred_2061)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_2061_c, y_train_2061)
y_pred_2061 = dt.predict(X_train_2061_c)
mean_squared_error(y_train_2061, y_pred_2061)**1/2

#### non-clustering features

##### Linear Support Vector Regressor from sklearn.svm

In [ ]:
regr = LinearSVR(random_state=123, tol=1e-5, loss='squared_epsilon_insensitive', fit_intercept=False, dual=False)
regr.fit(X_train_2061_nc, y_train_2061)
y_pred_2061 = regr.predict(X_train_2061_nc)
print(mean_squared_error(y_train_2061, y_pred_2061)**1/2)

##### Support Gradient Descent Regressor from sklearn.linear_model

In [ ]:
sgd = SGDRegressor(fit_intercept=False, max_iter=1000, random_state=123)
sgd.fit(X_train_2061_nc, y_train_2061)
y_pred_2061 = sgd.predict(X_train_2061_nc)
mean_squared_error(y_train_2061, y_pred_2061)**1/2

##### Lasso with Cross Validation from sklearn.linear_model

In [ ]:
lasso = LassoCV(fit_intercept=False)
lasso.fit(X_train_2061_nc, y_train_2061)
y_pred_2061 = lasso.predict(X_train_2061_nc)
mean_squared_error(y_train_2061, y_pred_2061)**1/2

##### Decision Tree Regressor from sklearn.tree

In [ ]:
dt = DecisionTreeRegressor(random_state=123)
dt.fit(X_train_2061_nc, y_train_2061)
y_pred_2061 = dt.predict(X_train_2061_nc)
mean_squared_error(y_train_2061, y_pred_2061)**1/2